# Chapter 6. Algorithms

**Deutsch Algorithm**

In [53]:
def is_balance_classic(f: callable) -> bool:
  o0 = f(0)
  o1 = f(1)
  if o0 == o1:
    return False
  return True

print("lambda x: 0 ➡️", is_balance_classic(lambda x: 0))
print("lambda x: x ➡️", is_balance_classic(lambda x: x))
print("lambda x: (x + 1) % 2 ➡️", is_balance_classic(lambda x: (x + 1) % 2))
print("lambda x: 1**x ➡️", is_balance_classic(lambda x: 1**x))

lambda x: 0 ➡️ False
lambda x: x ➡️ True
lambda x: (x + 1) % 2 ➡️ True
lambda x: 1**x ➡️ False


In [54]:
def build_matrix_from_function(f: callable):
  A = []
  for i in range(2):
    A.append([])
    for j in range(2):
      if f(j) == i:
        A[i].append(1)
      else:
        A[i].append(0)
  return A

print(build_matrix_from_function(lambda x: 0))
print(build_matrix_from_function(lambda x: x))
print(build_matrix_from_function(lambda x: (x + 1) % 2))
print(build_matrix_from_function(lambda x: 1**x))

[[1, 1], [0, 0]]
[[1, 0], [0, 1]]
[[0, 1], [1, 0]]
[[0, 0], [1, 1]]


In [55]:
def build_unitary_matrix_from_function(f: callable):
  A = []
  for i1 in range(2):
    for i2 in range(2):
      A.append([])
      for j1 in range(2):
        for j2 in range(2):
          if j1 == i1 and (f(j1) + j2) % 2 == i2:
            A[-1].append(1)
          else:
            A[-1].append(0)
  return A

print(build_unitary_matrix_from_function(lambda x: 0))
print(build_unitary_matrix_from_function(lambda x: x))
print(build_unitary_matrix_from_function(lambda x: (x + 1) % 2))
print(build_unitary_matrix_from_function(lambda x: 1**x))

[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]]
[[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
[[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]]


In [56]:
# Utility functions for matrix operations

def tensor_product(A: list[list[complex]], B: list[list[complex]]):
  res = []
  for i in range(len(A)):
    for j in range(len(B)):
      res.append([])
      for k in range(len(A[0])):
        for l in range(len(B[0])):
          res[i*len(B) + j].append(A[i][k]*B[j][l])
  return res

def multiply_matrixes(A: list[list[complex]], B: list[list[complex]]):
  ma = len(A)
  na = len(A[0])
  mb = len(B)
  nb = len(B[0])
  if na != mb:
    raise Exception("Inappropriate size!")
  res = []
  for i in range(ma):
    res.append([])
    for j in range(nb):
      sum = 0
      for k in range(na):
        sum += A[i][k]*B[k][j]
      res[i].append(sum)
  return res

def transpose_matrix(A: list[list[complex]]):
  AT = []
  for i in range(len(A[0])):
    AT.append([])
    for j in range(len(A)):
      AT[i].append(A[j][i])
  return AT

def perform_action(matrix, vector):
  return transpose_matrix(multiply_matrixes(matrix, transpose_matrix([vector])))[0]

In [57]:
def is_balance_quantum(F: list[list[complex]]):
  state0 = [0, 1, 0, 0] # |01>

  HH = tensor_product(
    [
      [0.5**0.5, 0.5**0.5],                                                                  
      [0.5**0.5, -0.5**0.5]
    ],
    [
      [0.5**0.5, 0.5**0.5],                                                                  
      [0.5**0.5, -0.5**0.5]
    ]
  )

  state1 = perform_action(HH, state0) # (|00> - |01> + |10> - |11>)/2
  
  state2 = perform_action(F, state1) # (-1^f(0)|0> + (-1)^f(1)|1>)(|0> - |1>)/2

  HI = tensor_product(
    [
      [0.5**0.5, 0.5**0.5],                                                                  
      [0.5**0.5, -0.5**0.5]
    ],
    [
      [1, 0],
      [0, 1]
    ]
  )

  state3 = perform_action(HI, state2) # (+-1)|0>(|0> - |1>)/√2 if F is constant or (+-1)|1>(|0> - |1>)/√2 if F is balanced

  print('\033[90m', f"The probabilities of the system in states |00>, |01>, |10>, |11> are {state3[0]**2}, {state3[1]**2}, {state3[2]**2}, {state3[3]**2}", '\033[0m', sep='')
  
  the_probabity_that_the_top_qubit_is_in_state_one = state3[2]**2 + state3[3]**2
  return abs(the_probabity_that_the_top_qubit_is_in_state_one - 1) < 1e-10 # the top qubit is in state |1>, conclusion: the function is balanced

print("lambda x: 0 ➡️", is_balance_quantum(build_unitary_matrix_from_function(lambda x: 0)))
print("lambda x: x ➡️", is_balance_quantum(build_unitary_matrix_from_function(lambda x: x)))
print("lambda x: (x + 1) % 2 ➡️", is_balance_quantum(build_unitary_matrix_from_function(lambda x: (x + 1) % 2)))
print("lambda x: 1**x ➡️", is_balance_quantum(build_unitary_matrix_from_function(lambda x: 1**x)))

The probabilities of the system in states |00>, |01>, |10>, |11> are 0.5000000000000002, 0.5000000000000002, 0.0, 0.0
lambda x: 0 ➡️ False
The probabilities of the system in states |00>, |01>, |10>, |11> are 0.0, 0.0, 0.5000000000000002, 0.5000000000000002
lambda x: x ➡️ True
The probabilities of the system in states |00>, |01>, |10>, |11> are 0.0, 0.0, 0.5000000000000002, 0.5000000000000002
lambda x: (x + 1) % 2 ➡️ True
The probabilities of the system in states |00>, |01>, |10>, |11> are 0.5000000000000002, 0.5000000000000002, 0.0, 0.0
lambda x: 1**x ➡️ False
